# Ejercicio 3 b. Traducción Automática de Texto

Este ejercicio tiene como objetivo entrenar un modelo para traducción automática de texto (neural machine translation) del inglés a español. Para ello, haremos uso de redes recurrentes y word embeddings. 

![neural machine translation](img/nmt.png)

## 1. Enunciado

La traducción de texto se suele realizar con modelos de tipo sequence-to-sequence, donde existe un *encoder* que codifica el lenguaje de entrada, y un *decoder* que genera el texto en el lenguaje de salida. Actualmente esto se realiza empleando redes con auto-atención (transformers), pero para este ejercicio vamos a implementar una red recurrente clásica. 

La implementación del modelo recurrente la puedes realizar basándote en los ejemplos:
1. [Modelo sequence-to-sequence a nivel de caracteres con LSTM](https://keras.io/examples/nlp/lstm_seq2seq/): Este ejemplo de Keras muestra cómo entrenar un modelo seq-to-seq implementado con LSTMs para la traducción de inglés a francés. Está basado en esta antigua entrada del [blog de Keras](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html), donde también se dan las pistas para trabajar a nivel de palabras.
2. [Traducción de inglés a español con un transformer](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/): Este ejemplo de Keras muestra como implementar un modelo seq-to-seq de tipo transformer en Keras, y cómo procesar el dataset de traducción de inglés al español con la capa `TextVectorization`.
3. [Traducción automática neuronal usando un modelo seq2seq a nivel de palabra](https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7): Este proyecto, cuyo código está disponible en este [repositorio de github](https://github.com/devm2024/nmt_keras), trabaja con un modelo seq-to-seq usando como tokens las palabras de las frases, para la traducción del inglés al francés. Incluye una capa de embedding vacía.

Tu trabajo consistirá en adaptar el código de los ejemplos anteriores para entrenar un modelo seq-to-seq basado en LSTMs para la traducción del inglés a español. Puedes tokenizar el texto con `Tokenizer` así como con `TextVectorization`, según te convenga mejor para construir las entradas. Sin embargo, debes utilizar una capa de word embedding pre-entrenada para inglés (Glove, Word2Vec...), como vimos en las prácticas. Es suficiente con entrenar tan solo un modelo de estas características.

*De forma opcional*, se valorará la comparativa del modelo obtenido con un modelo pre-entrenado de HuggingFace para la traducción de inglés al español con el dataset descargado. También se dará un punto extra se si usan métricas BLEU y ROUGE para comparar el rendimiento de los modelos.

**IMPORTANTE**: Se permiten cambios en el código para adaptarlo a la GPU empleada. Es posible que el modelo no se pueda cargar al completo en la GPU, por lo que se puede simplificar (usar un subconjunto más pequeño, un tamaño de batch más pequeño, etc.)

## 2. Entrega

La entrega de este ejercicio se realiza a través de la tarea creada para tal efecto en Enseñanza Virtual. Tienes que entregar un notebook, y el HTML generado a partir de él, cuyas celdas estén ya evaluadas.

La estructura del notebook debe contener los siguientes apartados:

0. Cabecera: nombre y apellidos.
1. Dataset: descripción, carga y procesado.
2. Selección y carga del word embedding para el inglés.
3. Modelo y configuración creadas en Keras y su entrenamiento. Debe incluir una explicación razonada de los componentes, y de la selección de valores como el número de unidades en las redes recurrentes (LSTM/GRU), dimensión del embedding, etc.
5. Análisis de resultados con comparativa respecto del trabajo original ([ejemplo 2](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/)) basado en transformers (*no es necesario mejorarlo*). Si se hace la parte opcional (comparar con un modelo pre-entrenado de HuggingFace), indicar la comparativa. El análisis puede ser cualitativo, haciendo pruebas de texto. *Se evaluará con 1 punto extra si se hace un análisis con métricas como BLEU y ROUGE (se pueden usar desde KerasNLP).*
6. Bibliografía utilizada (enlaces web, material de clase, libros, etc.).

### 2.1. Nota importante
-----
**HONESTIDAD ACADÉMICA Y COPIAS: un trabajo práctico es un examen, por lo que
debe realizarse de manera individual. La discusión y el intercambio de
información de carácter general con los compañeros se permite (e incluso se
recomienda), pero NO AL NIVEL DE CÓDIGO. Igualmente el remitir código de
terceros, OBTENIDO A TRAVÉS DE LA RED o cualquier otro medio, se considerará
plagio.** 

**Cualquier plagio o compartición de código que se detecte significará
automáticamente la calificación de CERO EN LA ASIGNATURA para TODOS los
alumnos involucrados. Por tanto a estos alumnos NO se les conservará, para
futuras convocatorias, ninguna nota que hubiesen obtenido hasta el momento.
SIN PERJUICIO DE OTRAS MEDIDAS DE CARÁCTER DISCIPLINARIO QUE SE PUDIERAN
TOMAR.**

-----

## 3. Código para iniciarse

En el [ejemplo 2](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/) indicado anteriormente, se puede ver cómo descargar y procesar un dataset de traducción del inglés al español. Abajo se deja igualmente la celda para descargar y cargar el dataset (se puede evaluar las veces que haga falta, ya que se descarga tan solo una vez, y se almacena en el directorio $HOME/.keras).

In [26]:
import pathlib
from keras import layers
from keras.layers import TextVectorization, Embedding, LSTM, Dense, Input
from tensorflow import keras

text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

El dataset viene en el siguiente formato: cada línea del fichero es una frase en inglés seguida por la correspondiente en español, separados por un tabulador. La siguiente celda separa cada frase en cada idioma, y además al español (idioma destino) le añade los tokens [start] y [end], necesarios para controlar la generación de la salida.

In [27]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

In [28]:
import random

for _ in range(5):
    print(random.choice(text_pairs))

('She always cries when she chops onions.', '[start] Siempre llora cuando pica cebolla. [end]')
("We're joking.", '[start] Estamos bromeando. [end]')
('I made fun of him.', '[start] Me burlé de él. [end]')
('When I was a child, I was spanked if I did something wrong.', '[start] Cuando era pequeño, me daban unos azotes cuando hacía algo malo. [end]')
('Come anytime.', '[start] Ven cuando quieras. [end]')


In [29]:
len(text_pairs)

118964

In [30]:
# Este código separa el conjunto de entrenamiento en train, val y test
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]
    
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


## 4. Modelos tipo seq-to-seq con Teacher Forcing

Un modelo de tipo sequence-to-sequence (seq-to-seq, o simplemente, seq2seq), se caracterizan porque reciben como entrada secuencias (texto) y generan como salida otra secuencia (texto). En nuestro caso la entrada será una frase en inglés y la salida será la frase en español.

Estos modelos se caracterizan porque están divididos en dos partes: un *encoder* y un *decoder*. Estos dos modelos se componen de la siguiente forma para conformar el modelo seq2seq (también conocido como *teacher forcing*):

![neural machine translation](img/seq2seq-teacher-forcing.png)


* El **encoder**:
  * **Recibe** la *secuencia de entrada* (frase en inglés). Cada token será una palabra, y se usará su representación con un word embedding pre-entrenado (Glove, Word2vec, FastText ...).
  * **Devuelve** el *estado oculto* de la última neurona de la red recurrente, que sirve como continuación para el decoder. Si es una LSTM, será el último hidden state y el cell state.
* El **decoder**:
  * **Recibe**:
    * El *último estado oculto (hidden state, cell state)* generado en el encoder.
    * La *secuencia de salida*, incluyendo el [start]. 
  * **Devuelve** la secuencia de salida desplazada en 1 posición. Si la frase original es "[start] Hablé con Tom [end]", la salida será "Hablé con Tom [end]".
  
La configuración del decoder es así porque se empleará en tiempo de inferencia de forma *auto-regresiva*; es decir: empezamos con tan solo "[start]", y el decoder generará la siguiente palabra (por ejemplo, "hablé"); esta palabra se concatena a la solución parcial, teniendo "[start] hablé"; se repite el proceso, le damos al decoder esa solución parcial y dará la siguiente palabra (por ejemplo, "con"), y la añadimos a la solución parcial "[start] hablé con", y así hasta alcanzar el token [end]. 
  
Recuerda que la salida del modelo indicará en formato one-hot cual es la siguiente palabra. Las entradas (del encoder y del decoder) serán las secuencias de los tokens en formato one-hot (que después pasarán por la correspondiente capa de embedding, siendo para el inglés un embedding pre-entrenado).


Let's start with vectorization. I use TextVectorization here, using the standard standardization for english words, and a custom one for spanish words (adding ¿ and without removing brackets.)

In [31]:
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
import string
import re 
import numpy as np
from keras.utils import to_categorical

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]

for i in range(len(train_pairs)):
    train_eng_texts[i] = train_eng_texts[i].lower()
    train_spa_texts[i] = train_spa_texts[i].lower()

test_eng_texts = [pair[0] for pair in test_pairs]
test_spa_texts = [pair[1] for pair in test_pairs]

val_eng_texts = [pair[0] for pair in val_pairs]
val_spa_texts = [pair[1] for pair in val_pairs]

for i in range(len(test_eng_texts)):
    test_eng_texts[i] = test_eng_texts[i].lower()
    test_spa_texts[i] = test_spa_texts[i].lower()
    val_eng_texts[i] = val_eng_texts[i].lower()
    val_spa_texts[i] = val_spa_texts[i].lower()

eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

eng_vocab = eng_vectorization.get_vocabulary()
spa_vocab = spa_vectorization.get_vocabulary()

Now we format the dataset. First we apply vectorization and then the dataset is formatted with encoder inputs (english sentences), decoder inputs (spanish sentences without the end), and the labels (spanish sentences displaced by one).

I tried to vectorize/tokenize and then encode with one-hot encoding, but since the vocabulary is big and we have a lot of sentences, every time that i tried i had a dead kernel error. It works if i reduce the sentences but then the accuracy is awful.

In [32]:
from keras.utils import pad_sequences

def format_dataset(data_eng, data_spa):
    return (
        {
            "encoder_inputs": data_eng,
            "decoder_inputs": data_spa[:, :-1],
        },
        data_spa[:, 1:]
    )

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    
    data_eng = eng_vectorization(np.array(eng_texts))

    data_spa= spa_vectorization(np.array(spa_texts))
    formatted_data = format_dataset(data_eng, data_spa)
    
    return formatted_data

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
test_ds = make_dataset(test_pairs)


Importing and creating an embedding matrix for Glove based on english sentences.

In [33]:
import os

glove_dir = 'glove.6B'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [34]:
embedding_dim = 100
embedding_matrix = np.zeros((len(eng_vocab), embedding_dim))
l= 0
for i,word in enumerate(eng_vocab):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Now we create the model. First the pre trained embedding layer, then a LSTM layer for the encoder. Same thing for the decoder but the embedding layer is not pre trained and the LSTM layer take as initial state, the final state of the encoder LSTM. The last layer is a softmax for classification.

In the embedding layer, since we have padded sequences, mask_zero=True is necessary.

In [35]:
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense
from keras import optimizers

encoder_inputs = Input(shape=(None,))

en_x= Embedding(input_dim=len(eng_vocab), output_dim=embedding_dim, mask_zero=True,weights=[embedding_matrix], trainable=False)(encoder_inputs)

encoder = LSTM(embedding_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))

dex=  Embedding(input_dim=len(spa_vocab),mask_zero=True, output_dim=embedding_dim)
final_dex= dex(decoder_inputs)

decoder_lstm = LSTM(embedding_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(units=len(spa_vocab), activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [36]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, None, 100)    1202900     ['input_7[0][0]']                
                                                                                                  
 embedding_6 (Embedding)        (None, None, 100)    1500000     ['input_8[0][0]']                
                                                                                            

In [37]:
history = model.fit(
    [train_ds[0]["encoder_inputs"], train_ds[0]["decoder_inputs"]],
    train_ds[1],
    epochs=100,
    batch_size=batch_size,
    validation_data=(
        [val_ds[0]["encoder_inputs"], val_ds[0]["decoder_inputs"]],
        val_ds[1]
    )
)


Epoch 1/100
1302/1302 [==============================] - 199s 151ms/step - loss: 6.1964 - acc: 0.1230 - val_loss: 5.2902 - val_acc: 0.1422
Epoch 2/100
1302/1302 [==============================] - 208s 160ms/step - loss: 5.1062 - acc: 0.2635 - val_loss: 4.8337 - val_acc: 0.2902
Epoch 3/100
1302/1302 [==============================] - 214s 164ms/step - loss: 4.8085 - acc: 0.2981 - val_loss: 4.6377 - val_acc: 0.3083
Epoch 4/100
1302/1302 [==============================] - 211s 162ms/step - loss: 4.6368 - acc: 0.3147 - val_loss: 4.4866 - val_acc: 0.3243
Epoch 5/100
1302/1302 [==============================] - 213s 164ms/step - loss: 4.4832 - acc: 0.3298 - val_loss: 4.3476 - val_acc: 0.3386
Epoch 6/100
1302/1302 [==============================] - 209s 160ms/step - loss: 4.3511 - acc: 0.3444 - val_loss: 4.2332 - val_acc: 0.3539
Epoch 7/100
1302/1302 [==============================] - 214s 164ms/step - loss: 4.2378 - acc: 0.3579 - val_loss: 4.1281 - val_acc: 0.3659
Epoch 8/100
1302/1302 [====

1302/1302 [==============================] - 223s 171ms/step - loss: 2.8357 - acc: 0.5579 - val_loss: 2.9138 - val_acc: 0.5375
Epoch 60/100
1302/1302 [==============================] - 224s 172ms/step - loss: 2.8312 - acc: 0.5592 - val_loss: 2.9072 - val_acc: 0.5393
Epoch 61/100
1302/1302 [==============================] - 222s 171ms/step - loss: 2.8266 - acc: 0.5606 - val_loss: 2.9052 - val_acc: 0.5409
Epoch 62/100
1302/1302 [==============================] - 223s 171ms/step - loss: 2.8220 - acc: 0.5616 - val_loss: 2.9020 - val_acc: 0.5407
Epoch 63/100
1302/1302 [==============================] - 221s 169ms/step - loss: 2.8176 - acc: 0.5629 - val_loss: 2.9002 - val_acc: 0.5425
Epoch 64/100
1302/1302 [==============================] - 222s 170ms/step - loss: 2.8139 - acc: 0.5642 - val_loss: 2.9003 - val_acc: 0.5422
Epoch 65/100
1302/1302 [==============================] - 222s 170ms/step - loss: 2.8112 - acc: 0.5652 - val_loss: 2.8967 - val_acc: 0.5430
Epoch 66/100
1302/1302 [=========

In [43]:
model.save("model_lstm_mask.h5")

In [44]:
input_token_index = {word: index for index, word in enumerate(eng_vocab)}
target_token_index = {word: index for index, word in enumerate(spa_vocab)}

eng_index_to_word = dict(enumerate(eng_vocab))
spa_index_to_word = dict(enumerate(spa_vocab))

Now, create the encoder and the decoder for making predictions from the trained model.

In [45]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

decoder_state_input_h = Input(shape=(embedding_dim,))
decoder_state_input_c = Input(shape=(embedding_dim,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_5 (Embedding)     (None, None, 100)         1202900   
                                                                 
 lstm_6 (LSTM)               [(None, 100),             80400     
                              (None, 100),                       
                              (None, 100)]                       
                                                                 
Total params: 1,283,300
Trainable params: 80,400
Non-trainable params: 1,202,900
_________________________________________________________________


Function for decoding the sequence. First it pass the input sequence to the encoder, then it starts generating the target sequence from '[start]' and from states values of encoder's lstm.

output_tokens will contain for each word of the sequence, the probability in the entire vocabulary.

In [48]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['[start]']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = spa_index_to_word[sampled_token_index]
        decoded_sentence += ' '+sampled_char

      
        if (sampled_char == '[end]' or len(decoded_sentence) > 100):
            stop_condition = True

        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [51]:
for _ in range(10):
    seq_index = random.randint(0, len(test_eng_texts))
    input_seq = test_ds[0]['encoder_inputs'][seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', test_eng_texts[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 9ms/step
-
Input sentence: ['tom is old.']
Decoded sentence:  tom es viejo [end]
1/1 [==============================] - 0s 9ms/step
-
Input sentence: ['what kind of movies do you like to watch?']
Decoded sentence:  qué tipo de te gusta ver a televisión [end]
1/1 [==============================] - 0s 10ms/step
-
Input sentence: ['the old castle lay in ruins.']
Decoded sentence:  la ciudad se [UNK] en la cama [end]
1/1 [==============================] - 0s 9ms/step
-
Input sentence: ["tom doesn't believe in ghosts."]
Decoded sentence:  tom no cree en la gente [end]
1/1 [==============================] - 0s 9ms/step
-
Input sentence: ['anything is infinitely better than nothing.']
Decoded sentence:  nada es más que un poco más de esto [end]
1/1 [==============================] - 0s 9ms/step
-
Input sentence: ["tom couldn't decide when to begin."]
Decoded sentence:  tom no se puede cuándo [UNK] [end]
1/1 [==============================] - 0s 10ms/s

We can see that the translation is absolutely not perfect, but in some cases is correct. I would say that maybe using a seq2seq with Transformer can bring to better results. 
Sometimes there is the [UNK] char because the spanish vocabulary was bigger than the max length set.